In [1]:
#ipython magic
%load_ext autoreload
%autoreload 2
%reset -f
%matplotlib notebook

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [2]:
#general imports
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time

In [3]:
# path to your vape repo
path_to_vape = '/home/jamesrowland/Documents/Code/Vape/'
sys.path.append(path_to_vape)
sys.path.append(os.path.join(path_to_vape, 'utils'))
path_to_vape

'/home/jamesrowland/Documents/Code/Vape/'

In [4]:
#notebook specific imports
from opto_stim_import2 import BlimpImport, OptoStim2p
import suite2p
from suite2p.run_s2p import run_s2p
from my_suite2p.settings import ops
from utils_funcs import get_tiffs
import re
import tifffile
import glob


In [5]:
mouse_id = 'RL048'
pkl_path='/home/jamesrowland/Documents/Code/Vape/run_pkls'

In [6]:
run_number = 23

In [7]:
def run_processor(mouse_id, run_number, pkl_path, reprocess=False, reload=False):
    
    mouse_folder = os.path.join(pkl_path, mouse_id)
    
    if not os.path.exists(mouse_folder):
        os.makedirs(mouse_folder)
    
    global pkl_file # im so sorry
    pkl_file = os.path.join(mouse_folder, 'run{}.pkl'.format(run_number))

    if os.path.isfile(pkl_file) and not reprocess:
        print('run number {} already processed\n'.format(run_number))
        
        if reload:
            with open(pkl_file, 'rb') as f:
                mouse_obj =  pickle.load(f)
            return mouse_obj
        else:
            return None
        
    mouse_obj = BlimpImport(mouse_id)
    try:
        mouse_obj.get_object_and_test(run_number, raise_error = True)
    except Exception as e:
        print('error for run number {}'.format(run_number))
        print('{}\n'.format(e))
        return None
        
    return mouse_obj

In [8]:
mouse_obj = run_processor(mouse_id, run_number, pkl_path, reprocess=True, reload=True)
mouse_obj

Importing data file: RL048-2019-08-02-182658.txt
pycontrol RL048-2019-08-02-182658 successfully matched to blimp folder 2019-08-02-182445
pycontrol RL048-2019-08-02-182658 rsync successfully matched to paq 2019-08-02_RL048_t003.paq


In [ ]:
tseries_paths = mouse_obj.tseries_paths

tseries_paths = [item for sublist in tseries_paths for item in sublist]
tseries_paths

In [ ]:
def get_num_frames(tiff_folder):
    
    '''takes input of list of tiff folders and returns
    number of frames in the first of each tiff folder'''
    
    with tifffile.TiffFile(tiff_folder) as tif:
        tif_tags = {}
        for tag in tif.pages[0].tags.values():
            name, value = tag.name, tag.value
            tif_tags[name] = value

    d = re.search('(?<=\[)(.*?)(?=\,)', tif_tags['ImageDescription'])
    n_frames = int(d.group(0))
    
    return n_frames

In [ ]:
num_frames = []

for tseries in tseries_paths:
    tiff = get_tiffs(tseries)
    assert len(tiff) == 1
    num_frames.append(get_num_frames(tiff[0]))
    
mouse_obj.num_frames = num_frames

In [ ]:
db = {
    
       'data_path': tseries_paths,                                         
       'diameter'  : 11, 
       'nplanes'  : mouse_obj.num_planes,
       'fs'        : 15, # string which specifies where the binary file will be stored (should be an SSD)   
}   

In [ ]:
# opsEnd=run_s2p(ops=ops,db=db)

In [ ]:
################################ delete this cell #############################################
# p = '/home/jamesrowland/Documents/packerstation/jrowland/Data/2019-07-09/2019-07-09_J048_t-003/suite2p/ops1.npy'
# opsEnd = np.load(p)

In [ ]:
save_path = opsEnd[0]['save_path0']
s2p_path  = os.path.join(save_path, 'suite2p')

In [ ]:
mouse_obj.s2p_path = s2p_path

In [12]:
with open(pkl_file, 'wb') as f:
    pickle.dump(mouse_obj, f)